# select significant electrodes for cue and read task

In [2]:
import numpy as np
from scipy import stats
import os
bands_list = {
    'delta': (1, 4),
    'theta': (4, 8),
    'alpha': (8, 12),
    'beta': (12, 30),
    'gamma': (30, 70),
    'high gamma':(70,150),
    'else2':(150,min(400/2, 300))
}
HS_list = [45,47,48,50,54,71,76,78,73]
freq_list = [400]

In [3]:
def load_band_data(path, HS, freq, band):
    lowfreq, highfreq = bands_list[band]
    block_data = np.load(os.path.join(path, f'HS{HS}/result/HS{HS}_Block_overt_covert_{int(lowfreq)}_{int(highfreq)}_{freq}hz.npy'), allow_pickle=True).item()
    # Filter and concatenate covert data
    covert_keys = [key for key in block_data.keys() if key.startswith('ECoG_covert_')]
    covert_data = np.concatenate([block_data[key] for key in covert_keys], axis=0)

    # Filter and concatenate overt data
    overt_keys = [key for key in block_data.keys() if key.startswith('ECoG_overt_')]
    overt_data = np.concatenate([block_data[key] for key in overt_keys], axis=0)

    return covert_data, overt_data
                
import numpy as np
from scipy import stats

def calculate_p_values(ecogReading_task):
    """
    计算每个电极在每个时间点的 p 值。
    参数:
    - ecogReading_task: numpy.ndarray, shape (n_trials, n_electrodes, n_times)
    返回:
    - p_values: numpy.ndarray, shape (n_electrodes, n_times)
    """
    # 获取形状信息
    n_trials, n_electrodes, n_times = ecogReading_task.shape

    # 定义基线时间窗(选取-0.2s-0s)
    start = int(1.5 * 400 - 0.2 * 400)
    end = start + 80
    p_values = np.zeros((n_electrodes, n_times))

    for electrode_idx in range(n_electrodes):
        electrode_data = ecogReading_task[:, electrode_idx, :]
        baseline = electrode_data[:, start:end].reshape(-1)
        for time_idx in range(400, n_times):
            current_data = electrode_data[:, time_idx]
            baseline_valid = baseline[~np.isnan(baseline)]
            current_data_valid = current_data[~np.isnan(current_data)]
            if len(current_data_valid) > 0 and len(baseline_valid) > 0:
                p_values[electrode_idx, time_idx] = stats.ttest_ind(
                    baseline_valid, current_data_valid, alternative='less'
                ).pvalue
            else:
                p_values[electrode_idx, time_idx] = np.nan
                
    return p_values

In [ ]:
base_path = '/public/DATA/covert_reading/HS_preanalysis_ECoG'  # 数据基础路径
save_base_path = '/public/DATA/covert_reading/t-test_results/'
for HS in HS_list:
    os.makedirs(os.path.join(save_base_path, f'HS{HS}'), exist_ok=True)
    band_covert_p_values, band_overt_p_values = {}, {}
    for band in bands_list.keys():
        covert_data, overt_data = load_band_data(base_path, HS, 400, band)
        covert_p_value = calculate_p_values(covert_data)
        overt_p_value = calculate_p_values(overt_data)
        band_covert_p_values[band] = covert_p_value
        band_overt_p_values[band] = overt_p_value

    np.save(os.path.join(save_base_path, f'HS{HS}/covert_allband_p_values.npy'), band_covert_p_values)
    np.save(os.path.join(save_base_path, f'HS{HS}/overt_allband_p_values.npy'), band_overt_p_values)
